# onboard new device(s) to nautobot
With the help of notebook script you can easily add devices to nautobot

## enter <font color='red'>username</font> and <font color='red'>password</font> to login to your devices

In [ ]:
username = "username"
password = "password"

## define your default settings
* if you want to add new devices but do not want to change existing devices set update_device to False
* if you want to add tags to the device and the interfaces set add_tags to True
* To import the primary Interface only set primary_only to True
* If you want to import the configs from a file set import_config to True
* If you update your device and you want to update the interfaces set update_interfaces to True

In [ ]:
update_device = False
add_tags = True
primary_only = True
import_config = False
update_interfaces = True
loglevel = 'INFO'

## import mandatory packages

In [ ]:
import os
import sys
import pandas as pd
from veritas.onboarding import onboarding
from loguru import logger
from veritas.tools import tools
from IPython.display import display, Markdown, JSON

## configure logger output

In [ ]:
logger.configure(extra={"extra": "unset"})
logger.remove()
logger.add(sys.stderr, level=loglevel)

## create onboarding instance

In [ ]:
onb = onboarding.Onboarding(username=username, password=password)

## You can either <font color='red'>read</font> the inventory from file or add a devicelist <font color='red'>manually</font>

### read inventory from file
supported file format are __xlsx, yaml and csv__

In [ ]:
devices = onb.read_inventory('./inventory.yaml')

or create your own list of devices (be careful: __this overwrites the devices above__)

In [ ]:
# devices = []
# devices.append({'name': 'lab.local', 'ip': '192.168.0.1'})

### if you want to see your devices run the next cell

In [ ]:
df = pd.DataFrame(devices)
df

Main loop: loop through all the devices

In [ ]:
for device in devices:
    hostname = device.get('name')
    logger.configure(extra={"extra": hostname})
    display(Markdown(f'*processing {hostname}*'))
    ip = device.get('ip', onb.get_ip_from_host(hostname))

    # we need the IP address of the host
    if not ip:
        display(Markdown(f'could not resolv IP address of {hostname}'))
        # if we import the config we set the IP to the hostname and import
        # the config and facts from file
        if import_config:
            ip = hostname
        else:
            continue

### check if host is in nautobot

In [ ]:
    device_in_sot = onb.device_in_sot(ip, hostname)
    display(Markdown(f'* {hostname} in sot: {device_in_sot}'))

### get device config, device facts and properties

#### first the default values

In [ ]:
device_defaults = onb.get_device_defaults(ip, device)

#### if you want to see the device defaults run the next cell

In [ ]:
display(JSON(device_defaults))

#### any changes? Than update device_defaults! (and rerun the cell above to see the changes!)

In [ ]:
# device_defaults.update({'status': {'name': 'Active'}})

#### get config and facts. Then parse config

In [ ]:
    device_config, device_facts = onb.get_device_config_and_facts(
                                    device_ip=ip, 
                                    device_defaults=device_defaults,
                                    import_config=import_config,
                                    import_filename=hostname)

    # parse config to get interfaces and so on
    parser = onb.parse_config(device_config, device_facts, device_defaults)

#### get primary address, device_properties, primary_interface and vlan_properties

In [ ]:
    primary_address = onb.get_primary_address()
    device_properties = onb.get_device_properties()
    primary_interface = onb.get_primary_interface(primary_address=primary_address, 
                                                  device_properties=device_properties)
    vlan_properties = onb.get_vlan_properties()

In [ ]:
display(JSON(primary_interface))

#### set interfaces
if primary_interface is true we create a list of a single interface
otherwise we get the interface properties from the config

In [ ]:
    if primary_only:
        interfaces = [{'name': primary_interface.get('name'),
                           'ip_addresses': [{'address': primary_interface.get('address'),
                                             'status': {'name': 'Active'}
                                            }],
                           'description': primary_interface.get('description','Primary Interface'),
                           'type': primary_interface.get('type', '1000base-t'),
                           'status': {'name': 'Active'}}]
    
    else:
        interfaces = onb.get_interface_properties()

#### if you want to look at the properties run the next cell

In [ ]:
display(JSON(device_properties))

#### want to change anything? Rerun the above cell to see the changes

In [ ]:
# device_properties.update({'status': {'name': 'Active'}})

#### if you want to see the interfaces run the next cells

In [ ]:
display(JSON(interfaces))

#### want to change anything? Rerun the above cell to see the changes

In [ ]:
# interfaces[0].update({'status': {'name': 'Active'}})

### add or update device to nautobot

In [ ]:
    if not device_in_sot:
        device_in_sot = onb.add_device_to_sot(
                        device_properties=device_properties,
                        primary_interface=primary_interface.get("name"),
                        interfaces=interfaces,
                        vlan_properties=vlan_properties,
                        add_prefix=False)
    elif update_device:
        onb.update_device_in_sot(device=device_in_sot,
                                 primary_address=primary_address,
                                 interfaces=interfaces,
                                 update_interfaces=update_interfaces, 
                                 primary_only=primary_only)

### If you want to add some tags .... go on
The device is now part of nautobot. If you want to add some tags to the device or 
the some of its interfaces go an.

In [ ]:
    if add_tags:
        device_fqdn = device_properties.get('device_fqdn', hostname)
        tag_properties = onb.get_tag_properties(device_fqdn,
                                                device_properties,
                                                device_facts)


#### if you want to look at the tag properties run the next cell¶

In [ ]:
display(JSON(tag_properties))

#### you can easily add some custom tags to the device or its interfaces.

In [ ]:
# tag_properties = [{'name': 'test', 
#                    'scope': 'dcim.device'},
#                   {'name': 'test', 
#                    'scope': 'dcim.interface',
#                    'interface': 'Loopback0'}]

### now add tags to nautobot

In [ ]:
        response = onb.add_tags(hostname=device_fqdn, 
                                tag_properties=tag_properties, 
                                device=device_in_sot)